## Here we have deployed the Convolutional Siamese model to verify signatures. You can use this to match your signatures or your friends to match against each other.

By - MOHD ASIF KHAN AND ABHISHEK KUMAR

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import ipywidgets as widgets
from torchvision import models
class SigNet(nn.Module):
    '''
    Reference Keras: https://github.com/sounakdey/SigNet/blob/master/SigNet_v1.py
    '''
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            #input size = [155, 220, 1]
            nn.Conv2d(1, 96, 11), # size = [145,210]
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, k=2, alpha=1e-4, beta=0.75),
            nn.MaxPool2d(2, stride=2), # size = [72, 105]
            nn.Conv2d(96, 256, 5, padding=2, padding_mode='zeros'), # size = [72, 105]
            nn.LocalResponseNorm(size=5, k=2, alpha=1e-4, beta=0.75),
            nn.MaxPool2d(2, stride=2), # size = [36, 52]
            nn.Dropout2d(p=0.3),
            nn.Conv2d(256, 384, 3, stride=1, padding=1, padding_mode='zeros'),
            nn.Conv2d(384, 256, 3, stride=1, padding=1, padding_mode='zeros'),
            nn.MaxPool2d(2, stride=2), # size = [18, 26]
            nn.Dropout2d(p=0.3),
            nn.Flatten(1, -1), # 18*26*256
            nn.Linear(18*26*256, 1024),
            nn.Dropout2d(p=0.5),
            nn.Linear(1024, 128),
        )


    def forward(self, x1, x2):
        x1 = self.features(x1)
        x2 = self.features(x2)
        return x1, x2

In [10]:
model = SigNet().to('cpu')

In [11]:
model.load_state_dict(torch.load("epoch_0_loss_0.449_acc_0.723.pt"))
model.eval()

SigNet(
  (features): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(1, 1))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout2d(p=0.3, inplace=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout2d(p=0.3, inplace=False)
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): Linear(in_features=119808, out_features=1024, bias=True)
    (14): Dropout2d(p=0.5, inplace=False)
    (15): Linear(in_features=1024, out_features=128,

In [12]:
import os
 
import torch
import torch.optim as optim
from PIL import ImageOps
from torchvision import transforms
from PIL import Image
import io

seed = 2020
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))
image_transform = transforms.Compose([
        transforms.Resize((155, 220)),
        transforms.ToTensor(),
        # TODO: add normalize
    ])

Device: cuda


In [13]:
btn_upload = widgets.FileUpload()
out_p1 = widgets.Output()
btn_upload_2 = widgets.FileUpload()
out_p2 = widgets.Output()
lbl_pred = widgets.Label()
btn_run =widgets.Button(description="Verify")

In [14]:
def on_click_verify(change):
    img = Image.open(io.BytesIO(btn_upload.data[-1]))
    out_p1.clear_output()
    with out_p1 : display(img)
    img_2 = Image.open(io.BytesIO(btn_upload_2.data[-1]))
    out_p2.clear_output()
    with out_p2 : display(img_2)
    x1 = Image.open(io.BytesIO(btn_upload.data[-1])).convert('L')
    x2 = Image.open(io.BytesIO(btn_upload_2.data[-1])).convert('L')
    x1 = image_transform(x1)
    x2 = image_transform(x2)
    x1 = torch.unsqueeze(x1,0)
    x2 = torch.unsqueeze(x2,0)
#     print(x1.shape,x2.shape)
    x1, x2 = model(x1, x2)
    distance = torch.pairwise_distance(x1, x2, p=2)
    if distance >= 0.04:
        pred = "Forged"
    else:
        pred = "Genuine"
    lbl_pred.value = "Dissimilarity {:.2f} Predicted is {} ".format(distance.item(),pred)
btn_run.on_click(on_click_verify)

In [15]:
widgets.VBox([widgets.Label("Select two signatures to verify!"),btn_upload,btn_upload_2,btn_run,out_p1,out_p2,lbl_pred])